In [1]:
!git clone https://github.com/slyucheng/144-Kaggle.git
!ls 144-Kaggle/

fatal: destination path '144-Kaggle' already exists and is not an empty directory.
gender_submission.csv test.csv              train.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [3]:
gender_submission = pd.read_csv('144-Kaggle/gender_submission.csv')
train = pd.read_csv('144-Kaggle/train.csv')
test = pd.read_csv('144-Kaggle/test.csv')

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# Data Preprocessing

In [7]:
def preprocessing(df):
    #First get rid of useless columns and perform one-hot.
    sex = pd.get_dummies(df.Sex, prefix='sex')
    df = pd.concat([df, sex], axis=1).drop(['Sex', 'Cabin', 'Ticket', 'Name'], axis=1)
    Embarkation = pd.get_dummies(df.Embarked, prefix='embarked')
    df = pd.concat([df, Embarkation], axis=1).drop('Embarked', axis=1)

    #Normalization
    scaler = StandardScaler()
    temp = pd.DataFrame(scaler.fit_transform(df[['Pclass', 'Age', 'Fare']])).rename(columns={0: 'Pclass', 1: 'Age', 2: 'Fare'})
    df = df.drop(['Pclass', 'Age', 'Fare'], axis=1)
    df = pd.concat([df, temp], axis=1)
    df = df.drop(['embarked_C', 'embarked_Q', 'embarked_S'], axis=1)
    df = df.fillna(df.mean())
    return df

processed_train = preprocessing(train)
processed_test = preprocessing(test)
display(processed_train.head())
display(processed_test.head())

,PassengerId,Survived,SibSp,Parch,sex_female,sex_male,Pclass,Age,Fare
0,1,0,1,0,0,1,0.827377,-0.530377,-0.502445
1,2,1,1,0,1,0,-1.566107,0.571831,0.786845
2,3,1,0,0,1,0,0.827377,-0.254825,-0.488854
3,4,1,1,0,1,0,-1.566107,0.365167,0.420730
4,5,0,0,0,0,1,0.827377,0.365167,-0.486337


,PassengerId,SibSp,Parch,sex_female,sex_male,Pclass,Age,Fare
0,892,0,0,0,1,0.873482,0.298549,-0.497811
1,893,1,0,1,0,0.873482,1.181328,-0.512660
2,894,0,0,0,1,-0.315819,2.240662,-0.464532
3,895,0,0,0,1,0.873482,-0.231118,-0.482888
4,896,1,1,1,0,0.873482,-0.584229,-0.417971


# Model Fitting

In [8]:
y = processed_train['Survived']
x = processed_train.drop(['PassengerId', 'Survived'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
x_train

,SibSp,Parch,sex_female,sex_male,Pclass,Age,Fare
114,0,0,1,0,0.827377,-8.748165e-01,-0.357308
440,1,1,1,0,-0.369365,1.054047e+00,-0.119886
625,0,0,0,1,-1.566107,2.156254e+00,0.002348
666,0,0,0,1,-0.369365,-3.237127e-01,-0.386671
348,1,1,0,1,0.827377,-1.839248e+00,-0.328280
...,...,...,...,...,...,...,...
562,0,0,0,1,-0.369365,-1.170488e-01,-0.376603
643,0,0,0,1,0.827377,2.174187e-16,0.489104
506,0,2,1,0,-0.369365,2.273911e-01,-0.124920
367,0,0,1,0,0.827377,2.174187e-16,-0.502864


In [9]:
clf = LogisticRegression(penalty='l2', max_iter=100)
clf = clf.fit(x_train, y_train)
y_pred_train = clf.predict(x_train)
y_pred_test = clf.predict(x_test)
print('Accuracy on training set using logistic regression: {}'.format(accuracy_score(y_train,y_pred_train)))
print('Accuracy on test set using logistic regression: {}'.format(accuracy_score(y_test,y_pred_test)))

Accuracy on training set using logistic regression: 0.8073836276083467
Accuracy on test set using logistic regression: 0.7835820895522388


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [10]:
clf = tree.DecisionTreeClassifier(max_depth=6)
clf = clf.fit(x_train, y_train)
y_pred_train = clf.predict(x_train)
y_pred_test = clf.predict(x_test)
print('Accuracy on training set using decision tree: {}'.format(accuracy_score(y_train,y_pred_train)))
print('Accuracy on test set using decision tree: {}'.format(accuracy_score(y_test,y_pred_test)))


Accuracy on training set using decision tree: 0.8667736757624398
Accuracy on test set using decision tree: 0.8022388059701493


In [11]:
clf = MLPClassifier(hidden_layer_sizes=(7, 21), max_iter=5000, 
                    solver='lbfgs', verbose=1)
clf = clf.fit(x_train, y_train)
y_pred_train = clf.predict(x_train)
y_pred_test = clf.predict(x_test)
print('Accuracy on training set using neural network: {}'.format(accuracy_score(y_train,y_pred_train)))
print('Accuracy on test set using neural network: {}'.format(accuracy_score(y_test,y_pred_test)))

Accuracy on training set using neural network: 0.9341894060995185
Accuracy on test set using neural network: 0.7835820895522388


In [12]:
param = {'max_depth':3, 
         'eta':0.5, 
         'objective':'binary:logistic'}
num_round = 100
dtrain = xgb.DMatrix(x_train, label=pd.DataFrame(y_train))
bst = xgb.train(param, dtrain, num_round)
# make prediction
#y_pred_train = bst.predict(dtrain)

dtrain = xgb.DMatrix(x_train)
y_pred_train = bst.predict(dtrain)
y_pred_train = [0 if y <0.5 else 1 for y in y_pred_train]


dtest = xgb.DMatrix(x_test)
y_pred_test = bst.predict(dtest)
y_pred_test = [0 if y <0.5 else 1 for y in y_pred_test]
#y_pred_test = bst.predict(x_test)
print('Accuracy on training set using XGboost: {}'.format(accuracy_score(y_train.tolist(),y_pred_train)))
print('Accuracy on test set using XGboost: {}'.format(accuracy_score(y_test.tolist(),y_pred_test)))
#print(ypred)




Accuracy on training set using XGboost: 0.9357945425361156
Accuracy on test set using XGboost: 0.8134328358208955


In [13]:
model = xgb.XGBClassifier(max_depth=3,learning_rate=0.03,n_estimators=500,objective='binary:logistic')
model.seed=42
kfold = KFold(n_splits=5)
results = cross_val_score(model, x, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 82.94% (2.31%)


In [14]:
neigh = KNeighborsClassifier(n_neighbors=5, algorithm='auto')
neigh.fit(x_train, y_train)
y_pred_train = neigh.predict(x_train)
y_pred_test = neigh.predict(x_test)

print('Accuracy on training set using KNN: {}'.format(accuracy_score(y_train,y_pred_train)))
print('Accuracy on test set using KNN: {}'.format(accuracy_score(y_test,y_pred_test)))

Accuracy on training set using KNN: 0.8571428571428571
Accuracy on test set using KNN: 0.7947761194029851


In [15]:
clf = svm.SVC(kernel='rbf')
clf = clf.fit(x_train, y_train)
y_pred_train = clf.predict(x_train)
y_pred_test = clf.predict(x_test)
print('Accuracy on training set using SVC: {}'.format(accuracy_score(y_train,y_pred_train)))
print('Accuracy on test set using SVC: {}'.format(accuracy_score(y_test,y_pred_test)))

Accuracy on training set using SVC: 0.8378812199036918
Accuracy on test set using SVC: 0.8134328358208955


/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [16]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
                             max_samples=0.5, max_features=0.5)
bagging = bagging.fit(x_train, y_train)
y_pred_train = bagging.predict(x_train)
y_pred_test = bagging.predict(x_test)
print('Accuracy on training set using bagging: {}'.format(accuracy_score(y_train,y_pred_train)))
print('Accuracy on test set using bagging: {}'.format(accuracy_score(y_test,y_pred_test)))

Accuracy on training set using bagging: 0.841091492776886
Accuracy on test set using bagging: 0.7873134328358209


In [17]:
from sklearn.ensemble import RandomForestClassifier
clf = BaggingClassifier(RandomForestClassifier(n_estimators=20, max_depth=5, random_state=42))
clf = clf.fit(x_train, y_train)
y_pred_train = clf.predict(x_train)
y_pred_test = clf.predict(x_test)
print('Accuracy on training set using RandomForest Ensemble: {}'.format(accuracy_score(y_train,y_pred_train)))
print('Accuracy on test set using RandomForest Ensemble: {}'.format(accuracy_score(y_test,y_pred_test)))

Accuracy on training set using RandomForest Ensemble: 0.8491171749598716
Accuracy on test set using RandomForest Ensemble: 0.8171641791044776
